In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [19]:
folder = "../../Data/hpc_out/rwm_wp_2_6r7m/"
mol_type = "6r7m"
#folder = "../../Data/collected_simulation_results/rwm_op_2_6r7m/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

acceptance_rates = Vector{Float64}([])
min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            id = parse(Int, split(file, separator)[id_index])
            exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
            exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
            theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
            string = "$(id): $(input["T"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
            push!(acceptance_rates, output["αs"][end])
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println(e)
            println("Error in $file")
        end
    end
end

In [20]:
println("Minimal configuration id: $(argmin(min_Es)) | Average acceptance rate: $(sum(acceptance_rates) / length(acceptance_rates))")

Minimal configuration id: 54 | Average acceptance rate: 0.2022564523486477


In [22]:
for elem in evaluation_strings
    println(elem)
end


2: 3.4 | 0.15223044 | 8138.976456129543, 8046.348161751665, 8084.221451586119 | 2.2122445893776446
3: 3.4 | 0.10455805 | 8125.515032380928, 8033.548109454436, 8036.822807349735 | 0.7915787897348405
4: 3.4 | 0.12186905 | 8270.168199312924, 8030.426381814662, 8089.358510309042 | 1.3285974745641378
5: 3.4 | 0.1840389 | 8119.111491421809, 8048.090407177127, 8092.686736835538 | 4.951360890422853
6: 3.4 | 0.23057178 | 8147.166036552964, 8043.737255273239, 8066.546086603208 | 4.957910662118398
7: 3.4 | 0.18065728 | 8113.906194337532, 8049.343003666172, 8092.221439347972 | 18.611214692330833
8: 3.4 | 0.19957484 | 8121.072121666841, 8049.501545715243, 8084.4458711553525 | 11.367127387309628
9: 3.4 | 0.19182171 | 8136.961861361987, 8049.688814064669, 8051.183662478754 | 19.690752160202333
10: 3.4 | 0.21719626 | 8153.929249311842, 8047.40667397829, 8056.670710665408 | 3.7375130063890274
11: 3.4 | 0.30850202 | 8311.622442593341, 8051.94483709321, 8100.736866557554 | 16.31886032156094
12: 3.4 | 0.

In [23]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(output["Es"])
        state = output["states"][mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
    catch e
        println(e)
        continue
    end
end

# Print Single Assembly Path

In [34]:
@load "../../Data/hpc_out/rwm_op_3_6r7m/5_rwm_op_3_6r7m.jld2" input output


2-element Vector{Symbol}:
 :input
 :output

In [35]:
input["template_centers"]

3×1206 Matrix{Float64}:
 -27.8796   -26.6674   -26.5597   …  -31.3903   -31.5191   -32.6934
   9.063      9.54921    9.03875       8.50467    8.25724    9.091
  -4.79258   -4.1493    -2.71995      -1.62113   -0.2168    -2.14403

In [36]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
